#### First look at the data

In [3]:
import pandas as pd

In [4]:
import json
rows = []
for l in open('../data/amazon_music/Digital_Music_5.json'):
    rows.append(json.loads(l))

df = pd.DataFrame(rows)
print(df.shape)
df.head()

(169781, 12)


,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,3,True,"06 3, 2013",A2TYZ821XXK2YZ,3426958910,{'Format:': ' Audio CD'},Garrett,"This is awesome to listen to, A must-have for ...",Slayer Rules!,1370217600,NaN
1,5.0,NaN,True,"10 11, 2014",A3OFSREZADFUDY,3426958910,{'Format:': ' Audio CD'},Ad,bien,Five Stars,1412985600,NaN
2,5.0,NaN,True,"02 11, 2014",A2VAMODP8M77NG,3426958910,{'Format:': ' Audio CD'},JTGabq,It was great to hear the old stuff again and I...,SLAYER!!!!!!!!!!!!!!!!!!!!!,1392076800,NaN
3,4.0,3,False,"12 7, 2013",AAKSLZ9IDTEH0,3426958910,{'Format:': ' Audio CD'},john F&#039;n doe,well best of's are a bit poison normally but t...,slayer greatest hits! you mean everything righ...,1386374400,NaN
4,5.0,NaN,True,"06 12, 2016",A3OH43OZJLKI09,5557706259,{'Format:': ' Audio CD'},melinda a goodman,What can I say? This is Casting Crowns!!!This ...,"This is a good, blessing filled",1465689600,NaN


In [6]:
df['overall'].value_counts()

5.0    135842
4.0     23143
3.0      6792
1.0      2192
2.0      1812
Name: overall, dtype: int64

In [7]:
df['reviewerID'].value_counts().min(), df['reviewerID'].value_counts().max()

(1, 791)

This dataset is meant to be pre-filtered to only include items/users with atleast 5 repeats, yet it seems a little mistakes slipped through. 
We'll enforce the filters ourselves.

In [8]:
df['asin'].value_counts().min(), df['asin'].value_counts().max()

(1, 574)

In [9]:
df['asin'].value_counts()

B00CZF8B68    574
B00BWGHIHY    316
B00136J7ZE    294
B00EH49FRE    283
B00136NUG6    262
             ... 
B000W1S2WU      1
B00125MGQI      1
B001QET6G8      1
B001664ADA      1
B00138APMQ      1
Name: asin, Length: 11797, dtype: int64

In [10]:
item_counts = df['asin'].value_counts()
user_counts = df['reviewerID'].value_counts()
items_to_remove = item_counts[item_counts < 5].index.tolist()
users_to_remove = user_counts[user_counts < 5].index.tolist()

filtered_df = df[(~df['asin'].isin(items_to_remove)) & (~df['reviewerID'].isin(users_to_remove))].copy()
print(filtered_df.shape, df.shape)

(168256, 12) (169781, 12)


In [11]:
# Is it actually legit to do filtering?
filtered_df = df.copy()

# Renaming for ease of matching to ml
filtered_df = filtered_df.rename(columns={'asin':'item_id', 'reviewerID':'user_id', 'overall':'rating'})

#### Moving to 0-based indexing

It would be easiest if both user ids and item ids were 0-indexed (e.g. for embedding layers later on, etc.)

In [12]:
user2idx = dict(zip(filtered_df['user_id'].unique().tolist(), range(filtered_df['user_id'].nunique())))

In [13]:
item2idx = dict(zip(filtered_df['item_id'].unique().tolist(), range(filtered_df['item_id'].nunique())))

In [14]:
filtered_df['user_id'] = filtered_df['user_id'].apply(user2idx.get)
filtered_df['item_id'] = filtered_df['item_id'].apply(item2idx.get)

In [15]:
filtered_df.head(10)

,rating,vote,verified,reviewTime,user_id,item_id,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,3,True,"06 3, 2013",0,0,{'Format:': ' Audio CD'},Garrett,"This is awesome to listen to, A must-have for ...",Slayer Rules!,1370217600,NaN
1,5.0,NaN,True,"10 11, 2014",1,0,{'Format:': ' Audio CD'},Ad,bien,Five Stars,1412985600,NaN
2,5.0,NaN,True,"02 11, 2014",2,0,{'Format:': ' Audio CD'},JTGabq,It was great to hear the old stuff again and I...,SLAYER!!!!!!!!!!!!!!!!!!!!!,1392076800,NaN
3,4.0,3,False,"12 7, 2013",3,0,{'Format:': ' Audio CD'},john F&#039;n doe,well best of's are a bit poison normally but t...,slayer greatest hits! you mean everything righ...,1386374400,NaN
4,5.0,NaN,True,"06 12, 2016",4,1,{'Format:': ' Audio CD'},melinda a goodman,What can I say? This is Casting Crowns!!!This ...,"This is a good, blessing filled",1465689600,NaN
5,4.0,NaN,True,"11 19, 2015",5,1,{'Format:': ' Audio CD'},Sylvia Arrowood,Enjoy Casting Crowns and their songs,Four Stars,1447891200,NaN
6,5.0,NaN,True,"06 6, 2015",6,1,{'Format:': ' Audio CD'},M Holley,Another very inspirational collection by a Ver...,Can't say enough. Great Christian music. God...,1433548800,NaN
7,5.0,NaN,False,"12 9, 2008",7,1,{'Format:': ' MP3 Music'},PacificHype,"I love the song Slow Fade by Casting Crowns, e...",DEFINITELY DESERVES PERFECT STARS!!!!,1228780800,NaN
8,5.0,NaN,False,"10 7, 2008",8,1,{'Format:': ' MP3 Music'},Sergio,This is an awesome album from the amazing Chri...,Can't go wrong with Casting Crowns,1223337600,NaN
9,5.0,NaN,False,"09 3, 2008",9,1,{'Format:': ' Audio CD'},Daniel T. Hughes,I can't believe I haven't written a review for...,"Great music, but even better if you see them l...",1220400000,NaN


In [16]:
filtered_df[['user_id', 'item_id']].isnull().max()

user_id    False
item_id    False
dtype: bool

In [17]:
filtered_df['user_id'].max(), filtered_df['user_id'].nunique()

(16565, 16566)

In [18]:
filtered_df['item_id'].max(), filtered_df['item_id'].nunique()

(11796, 11797)

#### Saving preprocessed dataset and preprocessing metadata

In [19]:
filtered_df[['user_id', 'item_id', 'rating']].to_csv('../data/am_preprocessed.csv', index=False)
json.dump({'user2idx':user2idx, 'item2idx':item2idx}, open('../data/am_preprocessing_metadata.json', 'w'))